## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,69.91,84,24,2.26,few clouds
1,1,Schladming,AT,47.3929,13.6870,35.53,99,99,5.14,light snow
2,2,Hervey Bay,AU,-25.2986,152.8535,83.03,78,100,12.50,overcast clouds
3,3,Kodiak,US,57.7900,-152.4072,38.71,100,100,9.22,mist
4,4,George,ZA,-33.9630,22.4617,74.10,76,15,1.66,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
vacation_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hervey Bay,AU,-25.2986,152.8535,83.03,78,100,12.50,overcast clouds
4,4,George,ZA,-33.9630,22.4617,74.10,76,15,1.66,few clouds
5,5,Bluff,NZ,-46.6000,168.3333,72.81,71,100,13.18,overcast clouds
10,10,Mataura,NZ,-46.1927,168.8643,73.47,66,100,6.82,light rain
17,17,Namibe,AO,-15.1961,12.1522,73.13,70,24,11.59,few clouds
18,18,Butaritari,KI,3.0707,172.7902,81.55,76,55,10.45,broken clouds
26,26,Castro,BR,-24.7911,-50.0119,75.72,72,4,4.79,clear sky
28,28,Georgetown,MY,5.4112,100.3354,82.33,86,20,2.30,few clouds
29,29,Makakilo City,US,21.3469,-158.0858,79.12,54,0,11.50,clear sky
30,30,Guiglo,CI,6.5456,-7.4966,74.66,28,100,2.04,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
vacation_cities_df.notnull().sum()

City_ID                276
City                   276
Country                275
Lat                    276
Lng                    276
Max Temp               276
Humidity               276
Cloudiness             276
Wind Speed             276
Current Description    276
dtype: int64

In [6]:
vacation_cities_df.count()

City_ID                276
City                   276
Country                275
Lat                    276
Lng                    276
Max Temp               276
Humidity               276
Cloudiness             276
Wind Speed             276
Current Description    276
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = vacation_cities_df.dropna(axis="index", how="any")

In [8]:
clean_df.count()

City_ID                275
City                   275
Country                275
Lat                    275
Lng                    275
Max Temp               275
Humidity               275
Cloudiness             275
Wind Speed             275
Current Description    275
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hervey Bay,AU,83.03,overcast clouds,-25.2986,152.8535,
4,George,ZA,74.10,few clouds,-33.9630,22.4617,
5,Bluff,NZ,72.81,overcast clouds,-46.6000,168.3333,
10,Mataura,NZ,73.47,light rain,-46.1927,168.8643,
17,Namibe,AO,73.13,few clouds,-15.1961,12.1522,
18,Butaritari,KI,81.55,broken clouds,3.0707,172.7902,
26,Castro,BR,75.72,clear sky,-24.7911,-50.0119,
28,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
29,Makakilo City,US,79.12,clear sky,21.3469,-158.0858,
30,Guiglo,CI,74.66,overcast clouds,6.5456,-7.4966,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hervey Bay,AU,83.03,overcast clouds,-25.2986,152.8535,Shelly Bay Resort
4,George,ZA,74.10,few clouds,-33.9630,22.4617,Protea Hotel by Marriott George King George
5,Bluff,NZ,72.81,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
10,Mataura,NZ,73.47,light rain,-46.1927,168.8643,Ellie's Villa
17,Namibe,AO,73.13,few clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
18,Butaritari,KI,81.55,broken clouds,3.0707,172.7902,Isles Sunset Lodge
26,Castro,BR,75.72,clear sky,-24.7911,-50.0119,CHACARA BAILLY
28,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
29,Makakilo City,US,79.12,clear sky,21.3469,-158.0858,Marriott's Ko Olina Beach Club
30,Guiglo,CI,74.66,overcast clouds,6.5456,-7.4966,Hotel GBAOLY 2


In [12]:
hotel_df.count()

City                   275
Country                275
Max Temp               275
Current Description    275
Lat                    275
Lng                    275
Hotel Name             275
dtype: int64

In [13]:
hotel_df.notnull().sum()

City                   275
Country                275
Max Temp               275
Current Description    275
Lat                    275
Lng                    275
Hotel Name             275
dtype: int64

In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]

In [25]:
clean_hotel_df.head(60)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hervey Bay,AU,83.03,overcast clouds,-25.2986,152.8535,Shelly Bay Resort
4,George,ZA,74.10,few clouds,-33.9630,22.4617,Protea Hotel by Marriott George King George
5,Bluff,NZ,72.81,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
10,Mataura,NZ,73.47,light rain,-46.1927,168.8643,Ellie's Villa
17,Namibe,AO,73.13,few clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
18,Butaritari,KI,81.55,broken clouds,3.0707,172.7902,Isles Sunset Lodge
26,Castro,BR,75.72,clear sky,-24.7911,-50.0119,CHACARA BAILLY
28,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
29,Makakilo City,US,79.12,clear sky,21.3469,-158.0858,Marriott's Ko Olina Beach Club
30,Guiglo,CI,74.66,overcast clouds,6.5456,-7.4966,Hotel GBAOLY 2


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.info_box_template = """
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}<dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.05)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))